In [267]:
import numpy as np
import math

def k(x, y, sig=1):
    r = np.linalg.norm(x - y)
    return np.exp(-0.5 * np.square(r) / np.square(sig))

def GaussianKernel(X, sig=1):
    n = X.shape[0]
    kernel = np.empty((n,n))
    for i in range(n):
        for j in range(n):
            kernel[i, j] = k(X[i], X[j], sig)
    return kernel / n
    # return kernel

def get_approximation(A, r):
    U, S, Vh = np.linalg.svd(A, full_matrices=True)
    return U[:, :r] @ np.diag(S[:r]) @ Vh[:r, :]

def NystromError(K, r):
    A = K[:r, :r]
    B = K[:r, r:]
    C = K[r:, r:]

    A_tilda = get_approximation(A, r)
    C_tilda = B.T @ np.linalg.pinv(A_tilda) @ B
    eps = np.linalg.norm(C - C_tilda, ord='fro')
    return eps / np.linalg.norm(C, ord='fro')

def average_orthog(K, r):
    A = K[:r, :r]
    B = K[:r, r:]
    C = K[r:, r:]
    L = []
    
    for i in range(B.shape[1]):
        for j in range(A.shape[1]):
            b_i = B[:, i].T
            a_j = A[:, j]
            length = np.linalg.norm(b_i) * np.linalg.norm(a_j)
            # L.append(abs(b_i @ a_j / length))
            theta = math.acos(b_i @ a_j / length)
            L.append(math.degrees(theta))
            
    return np.mean(L)

In [182]:
def sample_spherical(npoints, ndim=3):
        vec = np.random.randn(ndim, npoints)
        vec /= np.linalg.norm(vec, axis=0)
        return vec

def generate_sphere_data(n, d):
    phi = np.linspace(0, np.pi, 20)
    theta = np.linspace(0, 2 * np.pi, 40)
    x = np.outer(np.sin(theta), np.cos(phi))
    y = np.outer(np.sin(theta), np.sin(phi))
    z = np.outer(np.cos(theta), np.ones_like(phi))

    X = sample_spherical(n, d).T

    return X

In [274]:
# n, r, m = 100, 3, 10 actual variables

m, n, r = 100, 2, 10
X = generate_sphere_data(m, n)
# X = np.random.random((m, n))

K = GaussianKernel(X)
# K = X @ X.T

error = NystromError(K, r)
print(error)

# angle = average_orthog(K, r)
# print(angle)

# increasing n increases the error substantially.
# using spherical data increases error because kernel data is more orthog

0.002034741995668418
